In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
import pyomo.environ as pyo
from pyomo.environ import *
import sys


In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def generate_negatively_correlated_covariance_matrix(n):
    # Generate a random correlation matrix with negative correlations
    correlation_matrix = np.random.uniform(low=-0.9, high=-0.1, size=(n, n))
    np.fill_diagonal(correlation_matrix, 1)  # Set diagonal elements to 1
    correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2  # Ensure symmetry

    # Ensure the correlation matrix is valid for Cholesky decomposition
    cholesky_success = False
    while not cholesky_success:
        try:
            # Perform Cholesky decomposition
            cholesky_matrix = np.linalg.cholesky(correlation_matrix)
            cholesky_success = True
        except np.linalg.LinAlgError:
            # If Cholesky decomposition fails, adjust the correlation matrix slightly
            correlation_matrix += np.eye(n) * 0.01  # Adjust diagonal elements slightly

    # Generate random standard deviations
    #standard_deviations = np.random.uniform(0.5, 2, size=n)
    
    variance_outsideopt = np.random.uniform(30,40)
    variance_products = np.random.uniform(0.5,2,size=n-1)
    standard_deviations = np.hstack((variance_outsideopt,variance_products))
    # Construct diagonal matrix from standard deviations
    diagonal_matrix = np.diag(standard_deviations)
    #diagonal_matrix = np.eye(n)

    # Calculate the covariance matrix using Cholesky decomposition and diagonal matrix
    covariance_matrix = cholesky_matrix @ diagonal_matrix @ cholesky_matrix.T

    return covariance_matrix



In [5]:
def is_positive_semidefinite(matrix):
    """
    Check if a matrix is positive semidefinite.
    
    Args:
        matrix (numpy.ndarray): The matrix to check.
        
    Returns:
        bool: True if the matrix is positive semidefinite, False otherwise.
    """
    # Check if the matrix is symmetric
    if not np.allclose(matrix, matrix.T):
        return False
    
    # Check if all eigenvalues are non-negative
    eigvals, _ = np.linalg.eig(matrix)
    return np.all(eigvals >= 0)


In [6]:
def collection_distribution_prob_neg(n,collection):
    # generate mean 
    #mu = np.random.uniform(-2,2,n)
    mu_0 = np.random.uniform(6,10)
    mu_1n = np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    # generate covariance matrix with positive correlation
    neg_cov = generate_negatively_correlated_covariance_matrix(n)
    if is_positive_semidefinite(neg_cov): 
        sample_size = 10000
        utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    else:
        print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 
        
        
        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [7]:
def whole_instance_generation(n,whole_collection,whole_offertimes):
    ## generate LCMNL instances based on the assortment collection infomation
    # step 1: randomly generate LCMNL parameters
    # 1.1 the number of classes in LCMNL, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mnl
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCMNL instance with the above parameters of LCMNL
    true_instance = collection_distribution_prob_neg(n,whole_collection)
    # step 3: generate multinomial samples based on lcmnl instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [8]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [9]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [10]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [11]:
# check if the collection are nested
def check_subsets(lists):
    # Convert all lists to sets
    sets = [set(lst) for lst in lists]
    
    # Check subsets
    for i in range(len(sets) - 1):
        if not sets[i + 1].issubset(sets[i]):
            return False
    return True

# Check and print the result
result = check_subsets(all_full_assortment_index)
print("All subsequent lists are subsets of the previous one:", result)

All subsequent lists are subsets of the previous one: True


In [12]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [13]:
# generate instances that satisfy for assortment offertimes is >=20
## generate 50 random full instances 
# the observations with at least 20 times are >= 1e-3
pred_full_instance = []
pred_full_samples = []
#collection_20,offertimes_20,assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[0])

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)
    
    ''' condition = False
    for x in range(len(all_full_collections[0])):
        for y in all_full_collections[0][x]:
            if curr_choice_collection[y][x]<1e-11:
                condition = True
    while condition:
        curr_whole_instance,curr_purchase_samples = whole_instance_generation(jd_collection,jd_offertimes)
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

        condition = False
        for x in range(len(all_full_collections[0])):
            for y in all_full_collections[0][x]:
                if curr_choice_collection[y][x] < 1e-11:
                    condition = True
    if condition == True:
        print('instance_generation error')
    else:

            
        pred_full_instance.append(curr_whole_instance)
        pred_full_samples.append(curr_purchase_samples) '''

generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [14]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [15]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
       

        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
check assortment index for offertimes 30
check assortment index for offertimes 40
check assortment index for offertimes 50
check assortment index for offertimes 60


In [16]:
# train-test split
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(all_full_collections[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(all_full_collections[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(all_full_collections[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = all_full_instances[i][j][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = all_full_lb[i][j][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = all_full_ub[i][j][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(all_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(all_full_collections[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = all_full_instances[i][j][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        ''' df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst) '''
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        ''' df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''
        
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [88]:
def mnl_mle_pyo(data, collection,offertimes):
    n = data.shape[0]
    model = pyo.ConcreteModel()
    model.v = pyo.Var(range(n), bounds=(-50,50))
    
    def obj_rule(model):
        obj = 0
        for i in range(len(collection)):
            for j in collection[i]:
                obj += offertimes[i]*data[j][i] * model.v[j]
            obj -= offertimes[i]*pyo.log(sum([pyo.exp(model.v[k]) for k in collection[i]]))
        return obj
    
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)
    
    # scale the preference of the outside option to be 1
    model.outsideopt_constraint = pyo.Constraint(expr=model.v[0] == 0)
    
    opt = pyo.SolverFactory('ipopt')  # use iterior point method to solve convex optimizatio problem 
    results = opt.solve(model)
    
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        v_values = [model.v[i].value for i in range(n)]
        obj_value = model.obj()
        #model.pprint()
        return v_values, obj_value,results.solver.time
    else:
        raise ValueError("Solver failed to find an optimal solution.")

In [18]:
def mnl_distribution(v, collection):
    collection_distribution = np.zeros((len(v), len(collection)))
    for i, curr_collection in enumerate(collection):
        curr_total_v = np.sum([np.exp(v[j]) for j in curr_collection])
        for k, idx in enumerate(curr_collection):
            collection_distribution[idx][i] = np.exp(v[idx]) / curr_total_v
    
    return collection_distribution

In [19]:
def compute_collection(n):
    collection = []
    for i in range(n):
        subset = set(range(n)) - {i}
        collection.append(list(subset))
    return collection

In [20]:
compute_collection(3)

[[1, 2], [0, 2], [0, 1]]

In [21]:
def mccm_w_mnl(v_values,collection):
    probability_collection = mnl_distribution(v_values,collection)
    
    lam = np.zeros(len(v_values))
    rho = np.zeros((len(v_values),len(v_values)))
    u_values = np.zeros(len(v_values))
    ## compute u_values
    for i in range(len(v_values)):
        u_values[i] = np.exp(v_values[i])

    # compute lambda (initial probabilities)
    for i in range(len(v_values)):
        lam[i] = u_values[i]/np.sum(u_values)
        
    # compute transition matrix
    rho[0,0] = 1
    
    transition_collection = compute_collection(len(v_values))
    transition_collection_prob = mnl_distribution(v_values,transition_collection)
    for i in range(1,len(v_values)):
        rho[0,i] = 0
        for j in range(len(v_values)):
            if i !=j:
                rho[i,j] = (transition_collection_prob[j,i] - lam[j])/lam[i]
                if rho[i,j]<0:
                    print('transition matrix computation error')
                    sys.exit()
    
    return lam,rho,probability_collection

In [22]:
def mccm_mle_w_mnl_intial(data, collection,x_intial, lam_intial, rho_initial,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_mle_w_initial")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    for i in range(n):
        model.lam[i] = lam_intial[i]
        for j in range(m):
            model.x[i,j] = x_intial[i,j]
    
    for i in range(n):
        for j in range(n):
            model.rho[i,j] = rho_initial[i,j]

    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]* data[i, j] * pyo.log(model.x[i, j]) for i in collection[j] ) for j in range(m)), sense=pyo.maximize)
    #model.obj = Objective(expr=0,sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                #model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    #solver_options = {'max_iter': 15000, 'tol': 1e-4} 
    solver.options['max_iter'] = 10000
    #solver.options['tol'] = 1e-6
    
    try:
        # Your optimization solve code here
        #results = solver.solve(model)
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            y_values = [[model.y[i, j].value for j in range(m)] for i in range(n)]
            y_values = np.array(y_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values,lam_values,rho_values,y_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM MLE with MNL initial solution: No solution found or an error occurred.')
        #model.pprint()
        return [-100000, np.zeros((n,m)), np.zeros(n), np.zeros((n,n)), np.zeros((n,m)), -1]
    


In [23]:
def mccm_mle(data, collection,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    #model.abs_var = Var(range(n), range(m), within=NonNegativeReals, initialize=0, name="abs_val")
    
    for i in range(n):
        for j in range(n):
            if i != j:
                model.rho[i,j].setlb(0.00001) 
    
    for i in range(m):
        for j in collection[i]:
            model.x[j,i].setlb(0.00001)
    
    #model.obj = Objective(expr=sum(offer_times[j] * data[i, j] * model.abs_var[i, j] for i in range(n) for j in range(m)), sense=minimize)
    model.obj = pyo.Objective(expr= sum( sum(offertimes[j] * data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.maximize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                #model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                #model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
                #model.norm_constraint.add(model.x[j, i] == model.lam[j] + sum(model.y[k, i] * model.rho[k, j] for k in not_in_assortment[i]))
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                #model.norm_constraint.add(model.y[j, i] == model.lam[j] + sum(model.y[k, i] * model.rho[k, j] for k in not_in_assortment[i]))
                #model.norm_constraint.add(model.abs_var[j, i] == 0)
            
    
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    #solver.options['tol'] = 1e-9
    
    try:
        # Your optimization solve code here
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            
            y_values = [[model.y[i, j].value for j in range(m)] for i in range(n)]
            y_values = np.array(y_values)
            
            # Retrieve the solver runtime
            #model.pprint()
            return [pyo.value(model.obj), x_values, lam_values, rho_values, y_values, results.solver.time]
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM MLE with LB: No solution found or an error occurred.')
        #model.pprint()
        return [-100000, np.zeros((n,m)), np.zeros(n), np.zeros((n,n)), np.zeros((n,m)), -1]



In [24]:
def mccm_mle_nolb(data, collection,offertimes):
    model = pyo.ConcreteModel(name="mccm_mle")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals,bounds=(0.0001,1), name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals, name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    #model.abs_var = Var(range(n), range(m), within=NonNegativeReals, initialize=0, name="abs_val")
    
    #model.obj = Objective(expr=sum(offer_times[j] * data[i, j] * model.abs_var[i, j] for i in range(n) for j in range(m)), sense=minimize)
    model.obj = pyo.Objective(expr= sum( sum(offertimes[j] *data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.maximize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                #model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                #model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
                #model.norm_constraint.add(model.x[j, i] == model.lam[j] + sum(model.y[k, i] * model.rho[k, j] for k in not_in_assortment[i]))
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                #model.norm_constraint.add(model.y[j, i] == model.lam[j] + sum(model.y[k, i] * model.rho[k, j] for k in not_in_assortment[i]))
                #model.norm_constraint.add(model.abs_var[j, i] == 0)
            
    
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    
    try:
        # Your optimization solve code here
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            
            y_values = [[model.y[i, j].value for j in range(m)] for i in range(n)]
            y_values = np.array(y_values)
            
            # Retrieve the solver runtime
            #model.pprint()
            return [pyo.value(model.obj), x_values, lam_values, rho_values, y_values, results.solver.time]
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('Directly solve MCCM MLE: No solution found or an error occurred.')
        #model.pprint()
        return [-100000, np.zeros((n,m)), np.zeros(n), np.zeros((n,n)), np.zeros((n,m)), -1]



In [25]:
def mccm_mle_tol(data, collection,offertimes):
    model = pyo.ConcreteModel(name="mccm_mle_tol")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
            
    #model.obj = Objective(expr=sum(offer_times[j] * data[i, j] * model.abs_var[i, j] for i in range(n) for j in range(m)), sense=minimize)
    model.obj = pyo.Objective(expr= sum( sum(offertimes[j]* data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.maximize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)

    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
  

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    solver.options['tol'] = 1e-6
    #solver.options['print_level'] = 5
    
    
    try:
        # Your optimization solve code here
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            
            y_values = [[model.y[i, j].value for j in range(m)] for i in range(n)]
            y_values = np.array(y_values)
            
            # Retrieve the solver runtime
            
            #model.pprint()
            #print("Solver runtime:", runtime)
            #print('model objective', value(model.obj))
            return [pyo.value(model.obj), x_values, lam_values, rho_values, y_values, results.solver.time]
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM MLE with tolerence 1e-6: No solution found or an error occurred.')
        #model.pprint()
        return [-100000, np.zeros((n,m)), np.zeros(n), np.zeros((n,n)), np.zeros((n,m)), -1]


In [26]:
def mccm_revenue_prediction(lam, trans_matrix, assortment,price):
    # Model
    model = gp.Model()
    #model.Params.FeasibilityTol = 1e-4
    model.setParam('OutputFlag', 0)

    # Variables
    x = model.addVars(len(lam), vtype=GRB.CONTINUOUS,lb =0.0,name='x')
    y = model.addVars(len(lam), vtype=GRB.CONTINUOUS,lb=0.0, name='y')

    # Constraints
    model.addConstr(sum(x[i] for i in assortment)==1, 'normalization')
    
    not_in_assortment = []
    for i in range(len(lam)):
        if i not in assortment:
            not_in_assortment.append(i)
    #print('current assortment',assortment)
    #print('current compliment',not_in_assortment)

    for i in range(len(lam)):
        if i in assortment:
            model.addConstr(y[i] == 0, f'y_{i}_equals_0')
            model.addConstr(x[i] - lam[i] - sum(y[j] * trans_matrix[j][i] for j in not_in_assortment) == 0, f'balance_{i}')
        else:
            model.addConstr(x[i] == 0, f'x_{i}_equals_0')
            model.addConstr(y[i] - lam[i] -sum(y[j] * trans_matrix[j][i] for j in not_in_assortment) == 0, f'balance_{i}' )
            
    # Objective
    model.setObjective(sum(price[i]*x[i] for i in assortment), GRB.MAXIMIZE)
    #model.setObjective(0, GRB.MAXIMIZE)

    # Solve
    model.optimize()
    #print('MCCM LP solver status',model.status)
    if model.status == gp.GRB.OPTIMAL:
        # Extract variable values
        x_values = [x[i].x for i in range(len(lam))]

        # Check probability constraint
        if abs(sum(x_values) - 1) > 0.00001:
            print('Probability error')

        return model.objVal,x_values,model.Runtime

    if model.status == gp.GRB.INFEASIBLE:
        print("The model is infeasible.")
    if model.status == gp.GRB.UNBOUNDED:
        print("The model is unbounded.")
    # Check solution
    if model.status != GRB.OPTIMAL:
        print("Optimization failed to find an optimal solution")
        ''' model.computeIIS()
        infeas_constraints = [c.constrName for c in model.getConstrs() if c.IISConstr]
        print("The following constraints contribute to infeasibility:")
        print(infeas_constraints) '''
        return 0,np.zeros(n),-1

In [27]:
def mccm_prediction_tol(lam, trans_matrix, assortment,price):
    # Model
    
    model = gp.Model()
    model.Params.FeasibilityTol = 1e-3
    
    #model.setParam('DualReductions', 0)
    model.setParam('outputFlag',0)
    #model.setParam('BarHomogeneous', 1)

    #model.setParam('OutputFlag', 0)

    # Variables
    x = model.addVars(len(lam), vtype=GRB.CONTINUOUS,lb=0.0, name='x')
    y = model.addVars(len(lam), vtype=GRB.CONTINUOUS,lb=0.0, name='y')

    # Constraints
    model.addConstr(sum(x[i] for i in assortment)==1, 'normalization')
    
    not_in_assortment = []
    for i in range(len(lam)):
        if i not in assortment:
            not_in_assortment.append(i)
    #print('current assortment',assortment)
    #print('current compliment',not_in_assortment)

    for i in range(len(lam)):
        if i in assortment:
            model.addConstr(y[i] == 0, f'y_{i}_equals_0')
            model.addConstr(x[i] - lam[i] - sum(y[j] * trans_matrix[j][i] for j in not_in_assortment) == 0, f'balance_{i}')
        else:
            model.addConstr(x[i] == 0, f'x_{i}_equals_0')
            model.addConstr(y[i] - lam[i] -sum(y[j] * trans_matrix[j][i] for j in not_in_assortment) == 0, f'balance_{i}' )
    
    ''' for i in range(len(lam)):
        model.addConstr(x[i]+y[i] == lam[i] + sum(y[j]* trans_matrix[j][i] for j in range(len(lam))),f'balance_{i}' )
        if i in assortment:
            model.addConstr(y[i]==0, f'y_{i}_equals_0')
        else:
            model.addConstr(x[i]==0, f'x_{i}_equals_0') '''
            
    # Objective
    model.setObjective(sum(price[i]*x[i] for i in assortment), GRB.MAXIMIZE)
    #model.setObjective(0, GRB.MAXIMIZE)

    # Solve
    model.optimize()
    #print('MCCM LP solver status',model.status)
    #model.write("mccm_revenue.lp")
    if model.status == gp.GRB.OPTIMAL:
        # Extract variable values
        x_values = [x[i].x for i in range(len(lam))]

        # Check probability constraint
        if abs(sum(x_values) - 1) > 0.00001:
            print('Probability error')

        return model.objVal,x_values,model.Runtime
    
    ''' if model.status == gp.GRB.INFEASIBLE:
        print("The model is infeasible.")
        model.computeIIS()
        infeas_constraints = [c.constrName for c in model.getConstrs() if c.IISConstr]
        print("The following constraints contribute to infeasibility:")
        print(infeas_constraints)
    # Check dual information
        dual_values = model.getAttr("Pi", model.getConstrs())
        for constr, dual_value in zip(model.getConstrs(), dual_values):
            print(f"Dual value for constraint {constr.ConstrName}: {dual_value}")
    elif model.status == gp.GRB.UNBOUNDED:
        print("The model is unbounded.")
        # Check reduced costs
        reduced_costs = model.getAttr("RC", model.getVars())
        for var, reduced_cost in zip(model.getVars(), reduced_costs):
            print(f"Reduced cost for variable {var.VarName}: {reduced_cost}") '''

    # Check solution
    if model.status != GRB.OPTIMAL:
        print("Optimization failed to find an optimal solution")
        
        return 0,np.zeros(n),-1

In [28]:
def mccm_distribution(collection, lam_values, rho_values,price):
    prob = np.zeros((len(lam_values),len(collection)))
    revenue = []
    runtime = []
    
    for i in range(len(collection)):
        #prob[:,i]  = mccm_balance(lam_values, rho_values, collection[i])
        curr_assortment = collection[i]
        print(f'solve assortment {curr_assortment}')
        curr_rev,curr_prob,curr_runtime = mccm_revenue_prediction(lam_values, rho_values, collection[i],price)
        if curr_runtime <0:
            curr_rev,curr_prob,curr_runtime = mccm_prediction_tol(lam_values, rho_values, collection[i],price)
        print('optimal reveue is ',curr_rev)  
        prob[:,i] = curr_prob
        revenue.append(curr_rev)
        runtime.append(curr_runtime)
        #print('probability comparison:\n ',prob[:,i])
        #print(curr_prob)
    return revenue,prob,runtime

In [29]:
## prediction for the current testing collection of assortments 
def mccm_prediction(curr_test_instance,curr_test_collection,price,lam_values, rho_values):
    ''' true assortment ranking '''
    true_choice_prob = [] 
    true_revenue = []
    
    #mccm_revenue = []
    #mccm_sales = []
    
    #mccm_runtime = []
    
    
    for k in range(len(curr_test_collection)):

        # true revenue and true choice probabilities for current testing assortment
        curr_revenue = np.dot(curr_test_instance[:,k],price)
        true_revenue.append(curr_revenue)
        true_choice_prob.append(curr_test_instance[:,k])
        
    mccm_revenue,mccm_sales,mccm_runtime = mccm_distribution(curr_test_collection,lam_values,rho_values,price)
        
    return true_revenue,true_choice_prob,mccm_revenue,mccm_sales,mccm_runtime

In [30]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [31]:
## record representability of all instances

## record the kendall tau distance between the predicted ranking of mccm and the truth
all_mccm_distance_list = []  

## record the true revenue of the predicted best assortment
all_best_true_rev_list = []
all_best_rev_mccm_list = []  

## record the difference in best assortment
all_mccm_best_rev_diff = []

## record all the true and predicted revenues
all_true_revenue_list = []
all_mccm_revenue_list = []

all_true_prob_list = [] 
all_mccm_sales_list = []

all_true_ranking = []
all_mccm_ranking = []

## record the runtime of mccm prediction
all_mccm_pred_runtime = []

for i in range(len(all_full_collections)):
       
    # define as a container for quantity of the same collection size 
    # The following lists are of the same size as the instance size
    
    true_best_rev_collection = [] # a container for groundtruth best reveue 
    mccm_best_rev_collection = [] # a container for the best revenue predicted by robust mccm
    
    true_ranking_collection = [] # a container for groundtruth reveue ranking 
    mccm_ranking_collection = [] # a container for ranking by robust mccm
    
    mccm_distance_collection = [] # a container for kendall tau distance between true ranking and the ranking by robust mccm
   
    
    true_choice_prob_collection = []
    mccm_sales_frac_collection = []

    true_revenue_collection = []
    mccm_revenue_collection = []
    
    mccm_runtime_collection = []
    
    for j in range(pred_instance_size[i]):
        
        print(f'testing for offertimes {pred_offer_times_list[i]}, {j}th instance')
        
        ########### mccm prediction#################
        v_values, obj_value,runtime = mnl_mle_pyo(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j])
        mnl_lam,mnl_rho,initial_probability = mccm_w_mnl(v_values,all_train_collection[i][j])
        ll, choice_prob, lam_values, rho_values, y_values, ll_time = mccm_mle_w_mnl_intial(all_train_instances[i][j],all_train_collection[i][j],initial_probability,mnl_lam,mnl_rho,all_train_offertimes[i][j])
        
        if ll_time>0:
            true_revenue,true_choice_prob,mccm_revenue,mccm_sales,mccm_runtime = mccm_prediction(all_test_instance[i][j],all_test_collection[i][j],price,lam_values, rho_values)
        
        if ll_time<0 or np.any(np.array(mccm_runtime)<0):
            print('MLE:cannot solve MLE with mnl initial soluntion')
            ll, choice_prob, lam_values, rho_values, y_values, ll_time = mccm_mle(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j])
            if ll_time>0:
                true_revenue,true_choice_prob,mccm_revenue,mccm_sales,mccm_runtime = mccm_prediction(all_test_instance[i][j],all_test_collection[i][j],price,lam_values, rho_values)
        
        if ll_time<0 or np.any(np.array(mccm_runtime)<0):
            
            print(f'MLE:adjust mle lb ')
            
            ll, choice_prob, lam_values, rho_values, y_values, ll_time = mccm_mle_nolb(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j])
            if ll_time>0:
                true_revenue,true_choice_prob,mccm_revenue,mccm_sales,mccm_runtime = mccm_prediction(all_test_instance[i][j],all_test_collection[i][j],price,lam_values, rho_values)
        
        if ll_time<0 or np.any(np.array(mccm_runtime)<0):
            print(f'MLE:adjust tolerence parameters ')
            
            ll, choice_prob, lam_values, rho_values, y_values, ll_time = mccm_mle_tol(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j])
            if ll_time>0:
                true_revenue,true_choice_prob,mccm_revenue,mccm_sales,mccm_runtime = mccm_prediction(all_test_instance[i][j],all_test_collection[i][j],price,lam_values, rho_values)
        
        
    ### compute mle with mnl mle 
        ll_w_mnl = 0
        for x in range(len(all_train_collection[i][j])):
            for y in all_train_collection[i][j][x]:
                ll_w_mnl = ll_w_mnl + all_train_instances[i][j][y][x] * math.log(initial_probability[y][x])
        if ll < ll_w_mnl:
            lam_values = mnl_lam
            rho_values = mnl_rho
        if ll_time<0 or np.any(np.array(mccm_runtime)<0):
            true_revenue,true_choice_prob,mccm_revenue,mccm_sales,mccm_runtime = mccm_prediction(all_test_instance[i][j],all_test_collection[i][j],price,lam_values, rho_values)
        
        # the following lists are of the same size as number of testing assortments
        if np.any(np.array(mccm_runtime)<0):
            print('Prediction error to check')
            print('the current instances is offertimes {i} and instance {j}')
            sys.exit()
        
        total_runtime = []
        for x in range(len(mccm_runtime)):
            total_runtime.append(ll_time+mccm_runtime[x])
        
        true_revenue_collection.append(true_revenue)
        true_choice_prob_collection.append(true_choice_prob)
        
        mccm_revenue_collection.append(mccm_revenue)   
        mccm_sales_frac_collection.append(mccm_sales)
        mccm_runtime_collection.append(total_runtime)
         
        # true assortment ranking for test instance [i][j]
        curr_true_rank = np.argsort(-np.array(true_revenue))  
        true_ranking_collection.append(curr_true_rank)
        # true best assortment revenue for test instance [i][j]
        true_best_rev_collection.append(true_revenue[curr_true_rank[0]])
        
        # lb mdm assortment ranking for test instance [i][j] 
        curr_mccm_rank  = np.argsort(-np.array(mccm_revenue)) 
        mccm_ranking_collection.append(curr_mccm_rank)
        mccm_best_rev_collection.append(true_revenue[curr_mccm_rank[0]])
        
        
        #  mccm kendallTau Distance for current instance [i][j]
        curr_mccm_dist = kendall_tau_distance(curr_true_rank, curr_mccm_rank)
        mccm_distance_collection.append(curr_mccm_dist)
    
    
    # for each test instance [i][j] kendall tau distance is a number
    all_mccm_distance_list.append(mccm_distance_collection)
    
    # for each test instance [i][j] best revenue is a number
    all_best_true_rev_list.append(true_best_rev_collection) 
    all_best_rev_mccm_list.append(mccm_best_rev_collection)
    
    # for each test instance [i][j] best revenue difference is a number
    all_mccm_best_rev_diff.append((np.array(true_best_rev_collection) - np.array(mccm_best_rev_collection))/ np.array(true_best_rev_collection))
    
    # for each test instance [i][j], true revenue is a list of the same size as the number of testing assortments
    all_true_revenue_list.append(true_revenue_collection)
    
    all_mccm_revenue_list.append(mccm_revenue_collection)
    
    all_true_prob_list.append(true_choice_prob_collection)
    all_mccm_sales_list.append(mccm_sales_frac_collection)
 
    # for each test instance [i][j], true ranking is a list of the same size as the number of testing assortments
    all_true_ranking.append(true_ranking_collection)
    all_mccm_ranking.append(mccm_ranking_collection)

    ## for each test instance [i][j], runtime is a list of the same size as the number of testing assortments
    all_mccm_pred_runtime.append(mccm_runtime_collection)

testing for offertimes 20, 0th instance
solve assortment [0, 5]
Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic
optimal reveue is  0.3323555665662381
solve assortment [0, 3, 7]
optimal reveue is  0.1484435092249938
solve assortment [0, 5, 6]
optimal reveue is  0.2932774747998774
solve assortment [0, 5, 8]
optimal reveue is  0.41133779710468305
solve assortment [0, 6, 8]
optimal reveue is  0.30474786869074666
testing for offertimes 20, 1th instance
solve assortment [0, 3]
optimal reveue is  0.13801312016826917
solve assortment [0, 4]
Optimization failed to find an optimal solution
Changed value of parameter FeasibilityTol to 0.001
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
optimal reveue is  0.286823034204653
solve assortment [0, 1, 4]
Optimization failed to find an optimal solution
Changed value of parameter FeasibilityTol to 0.001
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
optimal reveue is  0.382780

In [32]:
## adding details 1
for i in range(len(pred_offer_times_list)):
    df_mccm_details1 = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mccm_dist':all_mccm_distance_list[i],
                                    'true_best_rev':all_best_true_rev_list[i],'mccm_best_rev':all_best_rev_mccm_list[i],
                                    'mccm_best_rev_diff':all_mccm_best_rev_diff[i]})
    df_mccm_details1.to_csv('prediction/mccm/revenue_prediction/details1/'+str(pred_offer_times_list[i])+'.csv')

In [33]:
## adding details 2 about ranking, revenue, choice probability
for i in range(len(pred_offer_times_list)): 
    for j in range(pred_instance_size[i]):
            true_rank = all_true_ranking[i][j]
            mccm_rank = all_mccm_ranking[i][j]
            df_rank = pd.DataFrame({'true_rank':true_rank,'mccm_rank':mccm_rank})
            df_rank.to_csv('prediction/mccm/revenue_prediction/details2/ranking/offertimes'+str(pred_offer_times_list[i])+'/rank_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
            
            true_rev = all_true_revenue_list[i][j]
            mccm_rev = all_mccm_revenue_list[i][j]
            df_rev = pd.DataFrame({'true_rev':true_rev,'mccm_rev':mccm_rev})
            df_rev.to_csv('prediction/mccm/revenue_prediction/details2/revenue/offertimes'+str(pred_offer_times_list[i])+'/rev_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
            
        
            mccm_runtime = all_mccm_pred_runtime[i][j]
            df_runtime = pd.DataFrame({'mccm_runtime':mccm_runtime})
            df_runtime.to_csv('prediction/mccm/revenue_prediction/details2/runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
            
            ## using sales fraction prediction
            ## record the choice probability prediction interval
            ''' true_prob = all_true_prob_list[i][j]
            df_true_prob = pd.DataFrame(true_prob)
            
            mccm_sales = all_mccm_sales_list[i][j]
            df_mccm_sales = pd.DataFrame(mccm_sales).T
    
            df_true_prob.to_csv('prediction/mccm/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
            df_mccm_sales.to_csv('prediction/mccm/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/mccm/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''

In [34]:
## avg prediction mccm runtime 
all_mccm_pred_avg_runtime = []

for i in range(len(pred_offer_times_list)):
    avg_mccm_pred_runtime_collection = []
    
    for j in range(pred_instance_size[i]): 
        avg_mccm_pred_runtime_collection.append(np.mean(np.array(all_mccm_pred_runtime[i][j]))) 
        
    all_mccm_pred_avg_runtime.append(np.mean(np.array(avg_mccm_pred_runtime_collection)))

In [35]:
avg_mccm_dist = []
avg_mccm_best_rev_diff = []

for i in range(len(pred_offer_times_list)):
    
    avg_mccm_dist.append(np.mean(all_mccm_distance_list[i]))
    avg_mccm_best_rev_diff.append(np.mean(all_mccm_best_rev_diff[i]))
    

df_mccm_jd_summary = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'avg_mccm_dist':avg_mccm_dist,'avg_mccm_best_rev_diff':avg_mccm_best_rev_diff,
                                   'avg_mccm_pred_runtime':all_mccm_pred_avg_runtime})
df_mccm_jd_summary.to_csv('prediction/mccm/jd_mccm_summary.csv')

In [36]:
df_mccm_jd_summary

,offertimes,avg_mccm_dist,avg_mccm_best_rev_diff,avg_mccm_pred_runtime
0,20,1.90,0.073108,10.352619
1,30,0.84,0.046535,10.605934
2,40,0.46,0.061864,8.496874
3,50,0.32,0.035253,6.834068
4,60,0.12,0.039283,7.451829


In [95]:
def mnl_mle_pyo(data, collection,offertimes):
    n = data.shape[0]
    model = pyo.ConcreteModel()
    model.v = pyo.Var(range(n), bounds=(-100,100))
    
    def obj_rule(model):
        obj = 0
        for i in range(len(collection)):
            for j in collection[i]:
                obj += offertimes[i]*data[j][i] * model.v[j]
            obj -= offertimes[i]*pyo.log(sum([pyo.exp(model.v[k]) for k in collection[i]]))
        return obj
    
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)
    
    # scale the preference of the outside option to be 1
    model.outsideopt_constraint = pyo.Constraint(expr=model.v[0] == 0)
    
    opt = pyo.SolverFactory('ipopt')  # use iterior point method to solve convex optimizatio problem 
    results = opt.solve(model)
    
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        v_values = [model.v[i].value for i in range(n)]
        obj_value = model.obj()
        #model.pprint()
        return v_values, obj_value,results.solver.time
    else:
        raise ValueError("Solver failed to find an optimal solution.")

In [96]:


def mccm_limit(data, collection,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_limit")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]* data[i, j] * model.abs_var[i, j] for i in collection[j] ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=-sum(data[i, j] * log(model.x[i, j]) for i in range(n) for j in range(m)),sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    #solver.options['tol'] = 1e-5
    
    
    try:
        # Your optimization solve code here
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
        #model.pprint()
        return [100000, np.zeros((n,m)), 0]
    
    
    


In [129]:

def mccm_limit_tol(data, collection,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_limit")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]* data[i, j] * model.abs_var[i, j] for i in collection[j] ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=-sum(data[i, j] * log(model.x[i, j]) for i in range(n) for j in range(m)),sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    solver.options['tol'] = 1e-6
    solver.options['dual_inf_tol'] = 1e-3  # Reduce tolerance for dual infeasibility
    solver.options['constr_viol_tol'] = 1e-8  # Tighten constraint violation tolerance
    
    
    try:
        # Your optimization solve code here
        results = solver.solve(model,tee=True)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
        #model.pprint()
        return [100000, np.zeros((n,m)), 0]


In [98]:
def mccm_limit_w_lb(data, collection,offertimes):
    
    model = pyo.ConcreteModel(name="mccm_limit_lb")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    for i in range(n):
        model.lam[i].setlb(1e-5)
        for j in range(n):
            if i != j:
                model.rho[i,j].setlb(1e-5)
    
    for i in range(m):
        for j in collection[i]:
            model.x[j,i].setlb(1e-5)
    
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]*data[i, j] * model.abs_var[i, j] for i in collection[j] ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=-sum(data[i, j] * log(model.x[i, j]) for i in range(n) for j in range(m)),sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    #solver.options['tol'] = 1e-5
    
    
    try:
        # Your optimization solve code here
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
        #model.pprint()
        return [100000, np.zeros((n,m)), 0]


In [103]:
def mccm_limit_w_intial(data, collection,offertimes,x_intial, lam_intial, rho_initial):
    
    model = pyo.ConcreteModel(name="mccm_limit")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    for i in range(n):
        model.lam[i] = lam_intial[i]
        for j in range(m):
            model.x[i,j] = x_intial[i,j]
    
    for i in range(n):
        for j in range(n):
            if i != j:
                model.rho[i,j] = rho_initial[i,j]
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]* data[i, j] * model.abs_var[i, j] for i in range(n) ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=0,sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                #model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    #solver_options = {'max_iter': 15000, 'tol': 1e-4} 
    solver.options['max_iter'] = 10000  
    
    try:
        # Your optimization solve code here
        #results = solver.solve(model)
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
        #model.pprint()
        return [100000, np.zeros((n,m)), 0]
    


In [102]:
def mccm_limit_w_intial_relax(data, collection,offertimes,x_intial, lam_intial, rho_initial):
    
    model = pyo.ConcreteModel(name="mccm_limit")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    for i in range(n):
        model.lam[i] = lam_intial[i]
        for j in range(m):
            model.x[i,j] = x_intial[i,j]
    
    for i in range(n):
        for j in range(n):
            if i != j:
                model.rho[i,j] = rho_initial[i,j]
    
    ''' for i in range(n):
        model.lam[i].setlb(1e-5)
        for j in range(n):
            if i != j:
                model.rho[i,j].setlb(1e-5)
    
    for i in range(m):
        for j in collection[i]:
            model.x[j,i].setlb(1e-5) '''
    
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]* data[i, j] * model.abs_var[i, j] for i in range(n) ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=0,sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                #model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    #solver_options = {'max_iter': 15000, 'tol': 1e-4} 
    solver.options['max_iter'] = 10000
    solver.options['tol'] = 1e-4
    solver.options['mu_strategy'] = 'adaptive'  # Change mu strategy
    solver.options['hessian_approximation'] = 'limited-memory'
    solver.options['dual_inf_tol'] = 1.0  # Adjust dual infeasibility tolerance
    solver.options['constr_viol_tol'] = 1e-4  # Adjust constraint violation tolerance   
    
    try:
        # Your optimization solve code here
        #results = solver.solve(model)
        results = solver.solve(model)
        status = str(results.solver.status)
        #print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
        #model.pprint()
        return [100000, np.zeros((n,m)), 0]
    


In [105]:
''' all_mccm_limit_loss = []
all_mccm_limit_prob = []
all_mccm_limit_runtime = []

 '''
## record the limit loss of all instances
all_mccm_limit_loss = []
all_mccm_limit_prob = []
all_mccm_limit_runtime = []

for i in range(len(pred_offer_times_list)):

    
    ## define container for mccm limit 
    mccm_limit_loss_collection =[]
    mccm_limit_runtime_collection =[]
    mccm_limit_probability_collection =[]
    
    for j in range(pred_instance_size[i]):
        print(f'testing mccm limit for offertimes {pred_offer_times_list[i]} instance {j}')
        
        v_values, obj_value,runtime = mnl_mle_pyo(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
        mnl_lam,mnl_rho,initial_probability = mccm_w_mnl(v_values,all_full_collections[i])
        mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime =mccm_limit_w_intial(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i],initial_probability,mnl_lam,mnl_rho)
        if mccm_limit_runtime == 0:
            mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime =mccm_limit_w_intial_relax(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i],initial_probability,mnl_lam,mnl_rho)
            if mccm_limit_runtime == 0:
                print('cannot solve with initial solution and try directly solve the mccm limit')
                mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime = mccm_limit(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])  
                
                if mccm_limit_runtime == 0:
                    print('cannot directly solve the mccm limit and try adding lb to solve')
                    mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime = mccm_limit_w_lb(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
                    
                    ''' if mccm_limit_runtime == 0: 
                        print('cannot solve the mccm limit by adding lb and try with accept with higher tolerence ')
                        mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime = mccm_limit_tol(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
                        
                        if mccm_limit_runtime == 0:
                            print('cannot solve with above 4 methods and accept MNL solution')
                            mccm_limit_prob = mnl_distribution(v_values,all_full_collections[i])
                            mccm_limit_runtime = runtime
                            
                            mccm_limit_loss = 0
                            for x in range(len(all_full_collections[i])):
                                for y in all_full_collections[i][x]:
                                    mccm_limit_loss = mccm_limit_loss + all_full_offertimes[i][x] * all_full_instances[i][j][y][x] * np.abs(all_full_instances[i][j][y][x] - mccm_limit_prob[y][x]) '''
        
        print(f'the limit of offertimes {pred_offer_times_list[i]} instance {j} is {mccm_limit_loss} with runtime {mccm_limit_runtime}')   
        mccm_limit_probability_collection.append(mccm_limit_prob)
        mccm_limit_loss_collection.append(mccm_limit_loss)
        mccm_limit_runtime_collection.append(mccm_limit_runtime)

    
    ## for each train instance [i][j] limit loss result is a number
    all_mccm_limit_loss.append(mccm_limit_loss_collection)
    all_mccm_limit_runtime.append(mccm_limit_runtime_collection)
    
    ## for each train instance [i][j] limit probability is a matrix
    all_mccm_limit_prob.append(mccm_limit_probability_collection)

testing mccm limit for offertimes 20 instance 0
the limit of offertimes 20 instance 0 is 41.454057606337706 with runtime 5.392797946929932
testing mccm limit for offertimes 20 instance 1
the limit of offertimes 20 instance 1 is 42.74139432761034 with runtime 10.082550048828125
testing mccm limit for offertimes 20 instance 2
the limit of offertimes 20 instance 2 is 44.01800907988928 with runtime 8.497473955154419
testing mccm limit for offertimes 20 instance 3
    model=mccm_limit;
        message from solver=Ipopt 3.12.12\x3a Maximum Number of Iterations
        Exceeded.
Error: Solver failed with status: warning
MCCM limit: No solution found or an error occurred.
the limit of offertimes 20 instance 3 is 41.637151501600364 with runtime 5.834878921508789
testing mccm limit for offertimes 20 instance 4
the limit of offertimes 20 instance 4 is 26.318956288294256 with runtime 9.637028932571411
testing mccm limit for offertimes 20 instance 5
the limit of offertimes 20 instance 5 is 45.74076

instance [3][14]


In [116]:
all_mccm_limit_loss[3][14], all_mccm_limit_prob[3][14] ,all_mccm_limit_runtime[3][14] = mccm_limit(all_full_instances[3][14],all_full_collections[3],all_full_offertimes[3])


12.366170136904191 [[6.65570175e-01 7.94871795e-01 7.23577236e-01 7.66666667e-01
  6.51817116e-01 6.80838672e-01 7.12454212e-01 6.22023701e-01
  6.65570176e-01 6.20967741e-01 6.60156250e-01 6.80838665e-01
  6.26865672e-01 6.33165829e-01 6.75521822e-01]
 [3.34429825e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.67022176e-14 2.45614044e-01
  1.96078437e-01 1.85483869e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.05128205e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.99220113e-13 0.00000000e+00
  1.86651305e-13 0.00000000e+00 6.25000000e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.76422764e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.95375406e-13 1.32362255e-01
  1.93180365e-13 0.00000000e+00 0.00000000e+00 1.65990590e-08
  0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+0

instance [3][40]

In [118]:
all_mccm_limit_loss[3][40], all_mccm_limit_prob[3][40] ,all_mccm_limit_runtime[3][40]  = mccm_limit(all_full_instances[3][40],all_full_collections[3],all_full_offertimes[3])


In [133]:
def mccm_limit_w_intial2(data, collection,offertimes,x_intial, lam_intial, rho_initial):
    
    model = pyo.ConcreteModel(name="mccm_limit")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    for i in range(n):
        model.lam[i] = lam_intial[i]
        for j in range(m):
            model.x[i,j] = x_intial[i,j]
    
    for i in range(n):
        for j in range(n):
            if i != j:
                model.rho[i,j] = rho_initial[i,j]
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j]* data[i, j] * model.abs_var[i, j] for i in range(n) ) for j in range(m)), sense=pyo.minimize)
    #model.obj = Objective(expr=0,sense=minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                #model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    #model.transition_constraints.add(sum(model.rho[0, j] for j in range(n)) == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] ==0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] =  10000
    solver.options['tol'] = 1e-6  # Tighten tolerance, might need further adjustments
    solver.options['dual_inf_tol'] = 1e-3  # Reduce tolerance for dual infeasibility
    solver.options['constr_viol_tol'] = 1e-8  # Tighten constraint violation tolerance
 
    
    try:
        # Your optimization solve code here
        #results = solver.solve(model)
        results = solver.solve(model,tee=True)
        status = str(results.solver.status)
        print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            #print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
        #model.pprint()
        return [100000, np.zeros((n,m)), 0]
    


instance [4][13]

In [134]:
v_values, obj_value,runtime = mnl_mle_pyo(all_full_instances[4][31],all_full_collections[4],all_full_offertimes[4])
mnl_lam,mnl_rho,initial_probability = mccm_w_mnl(v_values,all_full_collections[4])
mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime =mccm_limit_w_intial2(all_full_instances[4][31],all_full_collections[4],all_full_offertimes[4],initial_probability,mnl_lam,mnl_rho)
all_mccm_limit_loss[4][31] = mccm_limit_loss
all_mccm_limit_prob[4][31] = mccm_limit_prob 
all_mccm_limit_runtime[4][31] = mccm_limit_runtime

Ipopt 3.12.12: max_iter=10000
tol=1e-06
dual_inf_tol=0.001
constr_viol_tol=1e-08


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2586
Number of nonzeros in inequality constraint Jacobian.:      124
Number of nonzeros in Lagrangian Hessian.............:     1053

Total number of variables............................:      355
                     variables with only lower bounds:      355
                variables with lower and upper bounds:        0
 

instance [4][49]

In [139]:
all_mccm_limit_loss[4][49]

851.2363451910454

In [140]:
mccm_limit_loss, mccm_limit_prob,mccm_limit_runtime = mccm_limit(all_full_instances[4][49],all_full_collections[4],all_full_offertimes[4])
all_mccm_limit_loss[4][49] = mccm_limit_loss
all_mccm_limit_prob[4][49] = mccm_limit_prob
all_mccm_limit_runtime[4][49] = mccm_limit_runtime

1.9577933958664695

In [144]:
## adding limit results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    df_mccm_loss = pd.DataFrame({'ins_idx':ins_idx,'mccm_loss':all_mccm_limit_loss[i],'avg_loss':all_mccm_limit_loss[i]/sum(all_full_offertimes[i]),  'mccm_limit_time':all_mccm_limit_runtime[i]})
    df_mccm_loss.to_csv('limit/mccm/limit/'+str(pred_offer_times_list[i])+'.csv')
    
    for j in range(pred_instance_size[i]):
        df_limit_prob = pd.DataFrame(all_mccm_limit_prob[i][j])
        df_limit_prob.to_csv('limit/mccm/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [150]:
all_avg_loss = []
for i in range(len(pred_offer_times_list)):
    all_avg_loss.append(all_mccm_limit_loss[i]/sum(all_full_offertimes[i]))

In [151]:


avg_total_limit_loss = []
avg_toal_limit_loss_se = []
avg_mccm_limit_runtime = []

#avg_loss = []
avg_loss_se = []

for i in range(len(pred_offer_times_list)):

    
    avg_total_limit_loss.append(np.mean(all_mccm_limit_loss[i]))
    avg_mccm_limit_runtime.append(np.sum(all_mccm_limit_runtime[i]))
    avg_toal_limit_loss_se.append(np.std(all_mccm_limit_loss[i])/np.sqrt(len(all_mccm_limit_loss[i])))
    
    #avg_loss = np.mean(all_avg_loss[i])
    avg_loss_se.append(np.std(all_avg_loss[i])/np.sqrt(len(all_avg_loss[i])))

In [152]:
avg_loss = []
for i in range(len(avg_total_limit_loss)):
    avg_loss.append(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

In [153]:
df_mccm_limit = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mccm_loss':avg_total_limit_loss,'total_mccm_loss_se':avg_toal_limit_loss_se,
                                   'avg_mccm_loss':avg_loss,'avg_mccm_loss_se':avg_loss_se,
                                   'avg_mccm_limit_runtime':avg_mccm_limit_runtime
                                   })
df_mccm_limit.to_csv('limit/mccm/mccm_limit_summary.csv')
df_mccm_limit

,offertimes,total_mccm_loss,total_mccm_loss_se,avg_mccm_loss,avg_mccm_loss_se,avg_mccm_limit_runtime
0,20,36.715851,1.029740,0.004796,0.000135,599.083187
1,30,27.671420,0.919718,0.003675,0.000122,761.105921
2,40,17.989138,0.985292,0.002444,0.000134,514.440836
3,50,9.646535,0.834958,0.001342,0.000116,339.754596
4,60,6.467233,0.833142,0.000913,0.000118,466.141473


In [154]:
all_test_collection[0][0]

[[0, 5], [0, 3, 7], [0, 5, 6], [0, 5, 8], [0, 6, 8]]

In [155]:
for i in range(len(avg_total_limit_loss)):
    print(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

0.0047963228328896145
0.00367531141991045
0.002443844315344835
0.0013416599706696077
0.0009131930847690751
